In [6]:
import pandas as pd
import requests
import json

In [47]:
loc = 'C:\\Users\\Darrick\\Documents\\Jupyter\\senior-project\\stats\\'
season = '2020-21'

In [48]:
headers = {
    'Accept': 'application/json, text/plain, */*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://www.nba.com/'
}

# Retrieve both defensive dashboard and defensive impact
r_dd = requests.get('https://stats.nba.com/stats/leaguedashptdefend?College=&Conference=&Country=&DateFrom=&DateTo=&DefenseCategory=Overall&Division=&DraftPick=&DraftYear=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)
r_di = requests.get('https://stats.nba.com/stats/leaguedashptstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=PerGame&PlayerExperience=&PlayerOrTeam=Player&PlayerPosition=&PtMeasureType=Defense&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)

In [49]:
r_dd_dict = json.loads(r_dd.content)
r_di_dict = json.loads(r_di.content)

dd_df = pd.DataFrame(data=r_dd_dict['resultSets'][0]['rowSet'], columns=r_dd_dict['resultSets'][0]['headers'])
di_df = pd.DataFrame(data=r_di_dict['resultSets'][0]['rowSet'], columns=r_di_dict['resultSets'][0]['headers'])

dd_df.sort_values(by='PLAYER_NAME', inplace=True)
di_df.sort_values(by='PLAYER_NAME', inplace=True)

In [50]:
# Remove players not in both dataframes
diff = list(set(di_df['PLAYER_ID'].tolist()) - set(dd_df['CLOSE_DEF_PERSON_ID'].tolist()))

di_df = di_df[~di_df['PLAYER_ID'].isin(diff)]

# Combine dataframes
def_data = dd_df.join(di_df[di_df.columns[5:11]])

# Drop unnecessary columns
def_data.drop(columns=['PLAYER_LAST_TEAM_ID', 'PLAYER_LAST_TEAM_ABBREVIATION', 'PLAYER_POSITION', 'FREQ'], inplace=True)
def_data.rename(columns={'CLOSE_DEF_PERSON_ID' : 'PLAYER_ID'}, inplace=True)

def_data.head()

,PLAYER_ID,PLAYER_NAME,AGE,GP,G,D_FGM,D_FGA,D_FG_PCT,NORMAL_FG_PCT,PCT_PLUSMINUS,W,L,MIN,STL,BLK,DREB
172,203932,Aaron Gordon,25.0,50,50,4.64,10.08,0.460,0.471,-0.011,7.0,13.0,19.6,0.7,1.0,2.8
300,1628988,Aaron Holiday,24.0,65,65,3.60,7.74,0.465,0.452,0.013,22.0,21.0,31.8,1.1,0.3,3.6
380,1630174,Aaron Nesmith,21.0,38,38,2.74,6.03,0.454,0.458,-0.004,41.0,28.0,22.9,0.7,0.7,3.9
367,1627846,Abdel Nader,27.0,21,21,2.86,6.19,0.462,0.453,0.008,20.0,19.0,7.2,0.2,0.3,1.2
524,1629690,Adam Mokoka,22.0,6,6,0.67,1.83,0.364,0.465,-0.101,28.0,13.0,6.9,0.3,0.0,1.0


**Hustle Statistics**

In [51]:
r = requests.get('https://stats.nba.com/stats/leaguehustlestatsplayer?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=' + season + '&SeasonSegment=&SeasonType=Regular+Season&TeamID=0&VsConference=&VsDivision=&Weight=', headers=headers, timeout=10)

In [52]:
r_dict = json.loads(r.content)

df = pd.DataFrame(data=r_dict['resultSets'][0]['rowSet'], columns=r_dict['resultSets'][0]['headers'])
df.drop(columns=['TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'G', 'MIN', 'CONTESTED_SHOTS_2PT', 'CONTESTED_SHOTS_3PT',
                'SCREEN_AST_PTS', 'SCREEN_ASSISTS', 'OFF_LOOSE_BALLS_RECOVERED', 'DEF_LOOSE_BALLS_RECOVERED',
                'LOOSE_BALLS_RECOVERED', 'PCT_LOOSE_BALLS_RECOVERED_OFF', 'PCT_LOOSE_BALLS_RECOVERED_DEF',
                'OFF_BOXOUTS', 'BOX_OUTS', 'BOX_OUT_PLAYER_TEAM_REBS', 'BOX_OUT_PLAYER_REBS', 'PCT_BOX_OUTS_OFF',
                'PCT_BOX_OUTS_DEF', 'PCT_BOX_OUTS_TEAM_REB'], inplace=True)

# Remove players not in df
diff = list(set(df['PLAYER_ID'].tolist()) - set(def_data['PLAYER_ID'].tolist()))
df = df[~df['PLAYER_ID'].isin(diff)]

df.sort_values(by='PLAYER_NAME', inplace=True)

def_data = def_data.join(df[df.columns[2:]])
def_data['SEASON_ID'] = '220' + season[2:4]

def_data.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'AGE', 'GP', 'G', 'D_FGM', 'D_FGA',
       'D_FG_PCT', 'NORMAL_FG_PCT', 'PCT_PLUSMINUS', 'W', 'L', 'MIN', 'STL',
       'BLK', 'DREB', 'CONTESTED_SHOTS', 'DEFLECTIONS', 'CHARGES_DRAWN',
       'DEF_BOXOUTS', 'PCT_BOX_OUTS_REB', 'SEASON_ID'],
      dtype='object')

In [53]:
def_data.to_csv(loc + season + '_def_stats.csv', index=False)